Importing and logging into RH with enviroment variables for privacy.

In [20]:
import yfinance as yf
import robin_stocks.robinhood as r
import numpy as np
import pandas as pd
import time
import os
from sklearn.preprocessing import KBinsDiscretizer
username = os.getenv("ROBINHOOD_USERNAME")
password = os.getenv("ROBINHOOD_PASSWORD")
login = r.login(username,password)

ModuleNotFoundError: No module named 'sklearn'

RH data was not being kind, instead opted to use yfinance for historical data. Rolling mean of returns with a window of 24*4 = 96 (for 15-minute intervals, 24 hours).

In [2]:
doge_data = yf.download('DOGE-USD', interval='15m', period='1mo')
prices = doge_data['Close']

[*********************100%%**********************]  1 of 1 completed


Setting up Markov Chain. Defining multiple states based on volatility-adjusted thresholds, and creating transition matrix.

In [3]:
returns = prices.pct_change().dropna()
rolling_returns = returns.rolling(window=96).mean().dropna()
short_ma = prices.rolling(window=10).mean()  # 10-period moving average
long_ma = prices.rolling(window=50).mean()   # 50-period moving average
momentum = prices.diff(3)  # 3-period momentum
volatility = returns.rolling(window=10).std()
threshold = 2 * volatility.mean()

In [ ]:
features = pd.DataFrame({
    'Returns': returns,
    'Short_MA': short_ma,
    'Long_MA': long_ma,
    'Momentum': momentum,
    'Volatility': volatility
}).dropna()
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
states = est.fit_transform(features)
states_df = pd.DataFrame(states, columns=features.columns, index=features.index)

def calculate_transition_matrix(states_df):
    transitions = pd.crosstab(
        states_df.shift(1).apply(tuple, axis=1),
        states_df.apply(tuple, axis=1),
        normalize='index'
    )
    return transitions
transitions = calculate_transition_matrix(states_df)

Re-using Backtest framework from previous MDP project.

In [16]:
capital = 1000
dogecoin_holdings = 0
initial_buy_price = prices.iloc[0]
buy_and_hold_value = 1000 / initial_buy_price * prices

for i in range(len(states)-1):
    if rolling_returns.iloc[i] > 0 and capital > 0:  # Simplified buy condition
        dogecoin_holdings += capital / prices.iloc[i]
        capital = 0
    elif rolling_returns.iloc[i] < 0 and dogecoin_holdings > 0:  # Simplified sell condition
        capital += dogecoin_holdings * prices.iloc[i]
        dogecoin_holdings = 0

final_backtest_value = capital + dogecoin_holdings * prices.iloc[-1]
final_buy_and_hold_value = buy_and_hold_value.iloc[-1]

Print and Display Results.

In [17]:
print(f"Final Backtested Portfolio Value: ${final_backtest_value:.2f}")
print(f"Final Buy-and-Hold Portfolio Value: ${final_buy_and_hold_value:.2f}")

Final Backtested Portfolio Value: $1499.18
Final Buy-and-Hold Portfolio Value: $745.53
